In [1]:
import pandas as pd
import numpy as np

In [3]:
dbfile = '../clean/sfcompoDB_clean.csv'
fulldb = pd.read_csv(dbfile)

### Steal some functions/definitions from K. Dayman's sfcompo.py and refactor for my specific needs

In [91]:
# Conversion Factors
cF = {# Burnup, all to MWd/tUi
      'GW*d/tHMi': {'MW*d/tUi': 1000.0},
      'GW*d/tUi': {'MW*d/tUi': 1000.0},
      'MW*d/kgUi': {'MW*d/tUi': 1000.0},
      'MW*d/tHMi': {'MW*d/tUi': 1.0},
      'MW*d/tUi': {'MW*d/kgUi': 0.001, 'MW*d/tUi': 1.0},
      'MW*h/kgUi': {'MW*d/tUi': 41.66666667},
      # Isotopic Concentraion, to g/gUi or 
      'g/gFueli', 
      'mg/gFueli', 
      'mol/gFueli', 
      'mg/gUi',
      'g/gUi', 
      'μg/gUi',
      'g/gHMi',
      'mg/gHMi',
      'mg/kgUi',
      'g/kgUi',   
      'mol/molHMi', 
      'mg/tUi', 
      'kg/tUi', 
      'g/tUi',
      'g/tHMi', 
      'mol/molU238i',
      'g/gU238i',  
      'mol/molUi', 
      'g/gUf', 
      'mg/gUf', 
      'μg/gUf',  
      'kg/kgUf',    
      'mol/molUf', 
      'g/gU238f', 
      'mg/gU238f',
      'mol/molU238f', 
      'g/gPuf',
      'kg/kgPuf', 
      'mol/molPuf',
      # All activities to Ci/gUi or Ci/fUf
      'Bq/gUi': {'Ci/gUi': 2.7e-11}, 
      'Bq/kgUi': {'Ci/gUi': 2.7e-14},
      'Bq/gUf': {'Ci/gUf': 2.7e-11}, 
      'Ci/gFueli': {'Ci/gUi': 1.0}, 
      'Ci/gHMi': {'Ci/gUi': 1.0},
      # Misc
      'Pa': {'bar': 1e-05, 'psi': 0.000145038},
      'bar': {'Pa': 100000.0, 'psi': 14.5038},
      'cm': {'in': 0.393701, 'm': 0.01, 'mm': 10.0},
      'g': {'kg': 0.001, 'lb': 0.00220462, 'mg': 1000.0},
      'in': {'cm': 2.54, 'm': 0.0254, 'mm': 25.4},
      'kg': {'g': 1000.0, 'lb': 2.20462, 'mg': 1000000.0},
      'lb': {'g': 453.592, 'kg': 0.453592, 'mg': 453592.0},
      'm': {'cm': 100.0, 'in': 39.3701, 'mm': 1000.0},
      'mg': {'g': 0.001, 'kg': 1e-06, 'lb': 2.20462e-06},
      'mm': {'cm': 0.1, 'in': 0.0393701, 'm': 0.001},
      'psi': {'Pa': 689476.0, 'bar': 0.0689476}
      }

In [3]:
def changeUnits_(row, rv, ru, u, cF):
    '''
    Internal function used by changeUnits under Pandas.apply()
    '''
    cur_unit = u[int(row['idx'])]
    cf = getCF(cur_unit, ru, cF)
    row[rv] = row[rv] * cf
    return row


def changeUnits(dF, requestedValue, requestedUnit, U, cF):
    '''
    Given an SFCompo dF, the requestedValue (column) is extracted, and all
    values in that column are converted to the requestedUnit before returning.
    ConversionFactor table is queried and user is prompted as necessary.
    '''
    idx_col = dF.columns.get_loc(requestedValue)
    u = U[:, idx_col]
    dF['idx'] = np.arange(dF.shape[0])
    X = dF[['idx', requestedValue]].copy()
    X = X.apply(lambda x: changeUnits_(x, requestedValue, requestedUnit, u, cF), axis=1)
    X = X.drop('idx', axis=1)
    return X

In [4]:
def getUnit(s):
    '''
    Given a string, finds the numeric value and remainder, presumably a unit
    indication. Returns a float and a string.
    '''
    numeric = '0123456789-.'
    for i,c in enumerate(s):
        if c not in numeric:
            break
    number = s[:i]
    unit = s[i:].strip()
    return number, unit

def extractUnits(df, col):
    '''
    Iterates over column in a dataframe, extracting values and units.
    Returns a dataframe with two separated columns of values and units.
    '''
    units = []
    def f(x):
        value, unit = getUnit(x)
        units.append(unit)
        return value
    
    df[col] = df[col].apply(lambda x: f(x))
    df[col+'Units'] = units
    return df

In [5]:
fulldb.drop(columns=['Sample identifier', 'Z', 'A', 'I'], inplace=True)

In [6]:
cols = {'Reactor name' : 'ReactorName', 
        'Reactor type' : 'ReactorType',
        'Estimated burnup' : 'Burnup', 
        'e. U235' : 'Enrichment',
        'SFCompo sample ref' : 'SampleRef',
        'Measurement type' : 'Measurement', 
        'Item' : 'Isotope',
        'Value' : 'Value', 
        'Unit' : 'Unit',
        'Concentration' : 'Concentration', 
        'Concentration Unit' : 'ConcentrationUnit',
        'Sigma' : 'Sigma', 
        'Uncertainty' : 'Uncertainty'
        }

In [7]:
fulldb.rename(columns=cols, inplace=True)
fulldb = extractUnits(fulldb, 'Burnup')
fulldb = extractUnits(fulldb, 'Enrichment')

In [8]:
fulldb.Measurement.unique()

array(['Activity', 'Isotopic Concentration', 'Mass Ratio', 'Atom Ratio',
       'Burnup', 'Other', 'Activity Ratio', 'Depletion'], dtype=object)

In [14]:
conc = fulldb.loc[fulldb['Measurement'] == 'Isotopic Concentration'].copy()
units = conc.Unit.unique()
units

array(['g/gFueli', 'mg/gFueli', 'kg/tUi', 'mol/gFueli', 'mg/gUi',
       'mol/molUf', 'mol/molU238f', 'kg/kgUf', 'kg/kgPuf', 'mol/molPuf',
       'g/gUi', 'g/gU238f', 'g/tUi', 'mg/gUf', 'g/tHMi', 'mol/molU238i',
       'g/gHMi', 'mol/molUi', 'μg/gUi', 'μg/gUf', 'mg/gU238f', 'g/gU238i',
       'g/gUf', 'mol/molHMi', 'mg/kgUi', 'mg/tUi', 'g/kgUi', 'g/gPuf',
       'mg/gHMi'], dtype=object)

In [22]:
entries = []
for unit in units:
    tmp = conc.loc[conc.Unit == unit]
    entries.append(tmp.shape[0])
pretty = pd.DataFrame(zip(units, entries), columns=['Conc Unit', 'Num Entries'])
print(pretty.sort_values(by=['Num Entries'], ascending=False))

       Conc Unit  Num Entries
10         g/gUi         6928
4         mg/gUi         1946
11      g/gU238f         1740
15  mol/molU238i          699
16        g/gHMi          681
2         kg/tUi          473
5      mol/molUf          463
28       mg/gHMi          376
1      mg/gFueli          357
19        μg/gUf          261
18        μg/gUi          261
14        g/tHMi          209
26        g/kgUi          177
9     mol/molPuf          151
23    mol/molHMi          148
13        mg/gUf          137
12         g/tUi          110
22         g/gUf           93
0       g/gFueli           61
8       kg/kgPuf           45
7        kg/kgUf           36
6   mol/molU238f           34
20     mg/gU238f           34
3     mol/gFueli           25
24       mg/kgUi           17
17     mol/molUi           13
25        mg/tUi            6
27        g/gPuf            6
21      g/gU238i            1


In [24]:
pretty['Num Entries'].sum()

15488

In [116]:
### Other measurement types ###
act = fulldb.loc[fulldb['Measurement'] == 'Activity'].copy()
#mass = fulldb.loc[fulldb['Measurement'] == 'Mass Ratio'].copy()
#atom = fulldb.loc[fulldb['Measurement'] == 'Atom Ratio'].copy()
act.Unit.unique()

array(['Ci/gFueli', 'Bq/gUi', 'Bq/gUf', 'Ci/gUi', 'Ci/gUf', 'Ci/gHMi',
       'Bq/kgUi'], dtype=object)